In [1]:
#coding=gbk
import math
import random
import matplotlib.pyplot as plt
import time
import numpy as np
from decimal import *
getcontext().prec=36

class T_kernel_SGD():
    def __init__(self,d,s):
        self.d = d
        self.s = s
        self.Tn = [np.array([1.])]
        self.K_n = [[np.array([0.])]]
        self.K_t = [[ np.array([1.]) ]]
        self.L_n = 0
        self.fn = [[np.array([0.])]]
        self.f_average_n = [[np.array([0.])]]
        self.computing_ploy = []
        self.gamma0 = 1 #Initial step size
        self.t = 0 #Step decay rate
        self.epoch = 1 #  Current number of iterations
        self.theta = 1/(1+2*self.s)
        self.dim_Pi_Ln_d = 1
        self.cc_B4 = -math.pi**4/48
        
    def factorial(self,a,b,i,model):#i是间隔的步长,model是输出模式，输出列表或值
        if model == 'list':
            S = 1
            Output = []
            for k in range(0,b,i):
                S *= (k+a)
                Output.append(S)
        elif model ==  'value':
            S = 1
            for k in range(0,b,i):
                S *= (k+a)
            Output = S
        return Output
    
    def jiecheng(self,x):
        S = 1
        for i in range(1,x+1):
            S *= i
        return S
    
    def jiecheng_list(self,x):
        S = 1
        Output = [1]
        for i in range(1,x+1):
            S *= i
            Output.append(S)
        return Output
            
            
    
    def combination_num(self,a,b):#C^a_b
        if a == -1:
            C = 0
        elif a == 0:
            C = 1
        else:
            C = self.jiecheng(b)//(self.jiecheng(a)*self.jiecheng(b-a))
        return C
    
    def Multivariate_binomial(self,d,k,i):
        #self.T_n  is a polynomial before
        #k is the order of polynomial which will be calculating
        #d is dimension i in the element
        self.T_L_n = []
        if k == 1:
            self.T_L_n = np.array([1.])
        else:
            for a in self.Tn[i:]:
                self.T_L_n = np.append(self.T_L_n,a)
            Dim_list = [0]+[self.combination_num(l,d+l-1) for l in range(k)]
            for l in range(1,len(Dim_list)):
                for p in range(Dim_list[l-1],Dim_list[l]):
                    self.T_L_n[p] = self.T_L_n[p]*(k/(k-l+1))
        
        
    def updating_Zt(self):
        self.K_n = [[np.array([0.])]]
        if self.L_n >= 1:
            self.Tn = [np.array([1.])]
            omega_Ln = (self.dim_Pi_Ln_d)**(-2*self.s)
            fact1 = self.factorial(self.d/2,self.L_n,1,'value')#(d/2)k
            fact2 = [1]+self.factorial(2-self.L_n-self.d/2,int(self.L_n/2),1,'list')#(2-k-d/2)j
            fact3 = self.jiecheng_list(int(self.L_n/2))#(j)!
            fact4 = self.jiecheng_list(self.L_n)#(k-2j)!
            fact5 = [2**l for l in range(self.L_n+1)]
            #print("update Ln","1,",fact1,fact2,"3",fact3,fact4,self.L_n)
            Coe = [0 for l in range(self.L_n+1)]
            if self.L_n%2 == 0:
                for i in range(self.L_n+1):
                    if i%2 == 0:
                        Coe[i] = omega_Ln*fact1*fact5[i]/(fact2[-i//2-1]*fact3[-i//2-1]*fact4[i])
                    else:
                        Coe[i] = 0
            elif self.L_n%2 == 1:
                for i in range(self.L_n+1):
                    if i%2 == 1:
                        Coe[i] = omega_Ln*fact1*fact5[i]/(fact2[-(i+1)//2]*fact3[-(i+1)//2]*fact4[i])
                    else:
                        Coe[i] = 0
            #print(omega_Ln)
            #print(self.L_n,"Coe",Coe)            
            self.K_n[0][0][0] += Coe[0]*1   
            for k in range(1,self.L_n+1):
                self.Tn_lin = []
                for i in range(self.d):
                    self.Multivariate_binomial(self.d-i,k,i)
                    self.Tn_lin.append(self.T_L_n[:])
                self.Tn = self.Tn_lin[:]
                #print("L_n",self.L_n,k,self.Tn)
                self.K_n.append([ Coe[k]*A for A in self.Tn])
            self.K_t.append([np.zeros(len(A)) for A in self.Tn])
            self.K_t = [[a+b for a,b in zip(A,B)] for A,B in zip(self.K_t,self.K_n)]
            
    def Computation_ploynomial(self,x,order):#Note that here order is the order of the polynomial
        self.Tn = [np.array([1.])]
        if order == 0:
            self.computing_ploy = [self.Tn[:]]
        if order >= 1:
            self.computing_ploy = [self.Tn[:]]
            self.Tn = [ np.array([x[i]]) for i in range(self.d)]
            self.computing_ploy.append(self.Tn[:])
            for i in range(2,order+1):
                self.Tn_lin = []
                B = []
                for j in range(self.d-1,-1,-1):
                    B = np.append(self.Tn[j],B)
                    self.Tn_lin.insert(0,x[j]*B)
                self.Tn = self.Tn_lin[:]
                self.computing_ploy.append(self.Tn[:])
            
    def Computation_K_Xnt(self,X):
        self.Computation_ploynomial(X,self.L_n)
        self.K_Xnt_list = [[ a*b for a,b in zip(A,B)] for A,B in zip(self.computing_ploy,self.K_t)]     
    
    def Computation_fn(self,X):
        self.Computation_ploynomial(X,self.L_n)
        return sum([sum([ np.dot(a,b) for a,b in zip(A,B)])for A,B in zip(self.computing_ploy,self.fn)])
    
    def Computation_fn_average(self,X):
        self.Computation_ploynomial(X,self.L_n)
        return sum([sum([ np.dot(a,b) for a,b in zip(A,B)])for A,B in zip(self.computing_ploy,self.f_average_n)])
    
    def iteration(self,X,Y):
        coe = self.gamma0*((self.epoch)**(-self.t))*(Y - self.Computation_fn(X))
        if self.dim_Pi_Ln_d+1 < self.epoch**self.theta:
            self.L_n += 1 
            self.dim_Pi_Ln_d = self.combination_num(self.L_n,self.L_n+self.d-1)+self.combination_num(self.L_n-1,self.L_n+self.d-2)
            self.updating_Zt()
            A = [np.zeros(len(B)) for B in self.Tn]
            self.fn.append(A[:])
            self.f_average_n.append(A[:])
        self.Computation_K_Xnt(X)
        self.fn= [[ a+coe*b for a,b in zip(A,B)] for A,B in zip(self.fn,self.K_Xnt_list)] 
        self.f_average_n = [[ (self.epoch*a+b)/(self.epoch+1) for a,b in zip(A,B)] for A,B in zip(self.f_average_n,self.fn)]
    """    
    def construct_data(self):
        X = [random.gauss(0,1) for i in range(self.d)]
        mo = np.sqrt(sum([i**2 for i in X]))
        X = [i/mo for i in X]
        Y = np.sin(sum(X))
        Z = Y+random.gauss(0,0.05)
        return [X,Z,Y]
        """
    
    def construct_data(self):
        X = [random.gauss(0,1) for i in range(self.d)]
        mo = np.sqrt(sum([i**2 for i in X]))
        X = [i/mo for i in X]
        Y = 10*np.sin(X[0]*X[1])+20*(X[2]-0.5)**2+10*X[3]+5*X[4]+X[1]*X[2]*X[3]+np.cos(X[2]*X[3])
        Z = Y+random.gauss(0,0.2)
        return [X,Z,Y]

    '''
    def grho_cal(self,theta):
        t = theta/(2*math.pi)
        return (t**4-2*(t**3)+t**2-1/30)*self.cc_B4

    def construct_data(self):
        theta = 2*math.pi*random.random()
        Y = self.grho_cal(theta)
        Z = Y + random.gauss(0,0.5)
        return [[np.cos(theta),np.sin(theta)],Z,Y]
    '''
    
    def updating(self,gamma0,t,theta,Sum_epoch,NN):
        self.gamma0 = gamma0
        self.t = t
        self.theta = theta
        start = time.perf_counter()
        for i in range(1,Sum_epoch+1):
            self.epoch = i
            Z = self.construct_data()
            self.iteration(Z[0],Z[-2])
            if i in NN:
                S1 = 0
                S2 = 0
                end = time.perf_counter()
                for j in range(1000):
                    Z = self.construct_data()
                    S1 +=(self.Computation_fn(Z[0])-Z[-1])**2
                    S2 +=(self.Computation_fn_average(Z[0])-Z[-1])**2
                print('iteration:',i,S1/1000,S2/1000,self.L_n,end-start)
                #print(self.f_average_n)
                #print('K_n',self.K_n)
         

In [19]:
d = 2
N = [int(10**(i/5))for i in range(1,36)]
s = 1.
r=7/4
gamma0 =0.3
t = 0.0
theta = 1/(4*r*s+1)
Sum_epoch = 10000000
t_kernel_SGD = T_kernel_SGD(d,s)

t_kernel_SGD.updating(gamma0,t,theta,Sum_epoch,N)

iteration: 1 0.0020550003966044437 0.00204426746095485 0 0.00011170003563165665
iteration: 2 0.00971484789727472 0.0028227988124713194 0 0.02448660001391545
iteration: 3 0.06496499152645109 0.008669006800532529 0 0.052375400031451136
iteration: 6 0.07242899150029547 0.016434169815112743 0 0.07709760003490373
iteration: 10 0.003226723721572014 0.005976132256353122 0 0.10486610000953078
iteration: 15 0.0020546178250282375 0.0026374146790633995 0 0.13220340001862496
iteration: 25 0.001958298460783417 0.00563367181611627 0 0.15801600000122562
iteration: 39 0.13171848935496078 0.0057830685677070295 0 0.18166420003399253
iteration: 63 0.012116251325986918 0.005036405023800025 0 0.20294810004997998
iteration: 100 0.03246358119571217 0.0022033455681153558 0 0.22474350000265986
iteration: 158 0.0021206385213986415 0.0027097190794489598 0 0.2487053000368178
iteration: 251 0.005643056402006911 0.0020920848526448984 0 0.27507060003699735
iteration: 398 0.016572838056573804 0.0022079735014668613 1 

In [12]:
d = 5
N = [int(10**(i/5))for i in range(1,36)]
s = 0.6
gamma0 =0.5
t = 0.05
theta = 1/(2*s+1)
Sum_epoch = 10000000
t_kernel_SGD = T_kernel_SGD(d,s)

t_kernel_SGD.updating(gamma0,t,theta,Sum_epoch,N)

iteration: 1 188.04848842599822 206.51144553933113 0 0.00015490001533180475
iteration: 2 222.16934328455028 200.19466851270252 0 0.02071889999206178
iteration: 3 163.88981235244566 190.3562650554711 0 0.04347950001829304
iteration: 6 108.63683850061614 159.47411456923317 1 0.06029670001589693
iteration: 10 178.45065394708823 141.8175092864933 1 0.09480630001053214
iteration: 15 132.9458010034279 103.3224834211267 1 0.12918240000726655
iteration: 25 51.10687400280533 70.9470972946382 1 0.16291390001424588
iteration: 39 27.118060644539394 44.23779491370182 1 0.19852629999513738
iteration: 63 27.615905685369217 27.819816840392257 1 0.23855800001183525
iteration: 100 16.5248866844717 20.577780703164954 2 0.27962690001004376
iteration: 158 10.622252153015886 17.26345627827419 2 0.3745221999997739
iteration: 251 1.106451121699061 9.143897708306545 2 0.4671264000062365
iteration: 398 0.164901535373556 4.224468370118739 2 0.5645496000070125
iteration: 630 0.021738557347838933 1.736134891324326

KeyboardInterrupt: 

In [8]:
d = 20
N = [int(10**(i/5))for i in range(1,36)]
s = 0.6
gamma0 =0.5
t = 0.05
theta = 1/(2*s+1)
Sum_epoch = 10000001
t_kernel_SGD = T_kernel_SGD(d,s)

t_kernel_SGD.updating(gamma0,t,theta,Sum_epoch,N)

iteration: 1 48.594154638021955 63.9710474554201 0 0.00014469993766397238
iteration: 2 43.88509405981403 52.79263249587472 0 0.051086299936287105
iteration: 3 28.09450845472249 39.78968417750567 0 0.09241799998562783
iteration: 6 41.61437655164839 25.540799829358654 1 0.1388833998935297
iteration: 10 28.723914307815875 27.995552853815425 1 0.286161599913612
iteration: 15 24.65638754059362 26.099831178253577 1 0.43450019997544587
iteration: 25 28.35501923588623 24.700235050208835 1 0.6146984999068081
iteration: 39 20.124773611106036 23.142058365183875 1 0.7540197999915108
iteration: 63 16.926685801128816 22.78764005624148 1 0.939933699904941
iteration: 100 8.698541002357972 16.72236743921238 1 1.124883099924773
iteration: 158 5.2187111806143 11.762632490467432 1 1.305873499950394
iteration: 251 3.7389970677251014 8.476211310475747 1 1.4933658998925239
iteration: 398 2.4005136986069515 5.441672052039581 1 1.6853938999120146
iteration: 630 1.9092582359600052 3.141474636053255 1 1.90446399

In [13]:
d = 20
N = [int(10**(i/5))for i in range(1,36)]
s = 0.6
gamma0 =0.6
t = 0.055
theta = 1/(2*s+1)
Sum_epoch = 10000001
t_kernel_SGD = T_kernel_SGD(d,s)

t_kernel_SGD.updating(gamma0,t,theta,Sum_epoch,N)

iteration: 1 30.155595831725147 39.60652804914418 0 0.0001281000440940261
iteration: 2 41.52609571712632 30.03490348006633 0 0.06831889995373785
iteration: 3 37.18348963280113 27.334804865513686 0 0.11244389996863902
iteration: 6 27.048032761915067 27.348201418655474 1 0.15485119994264096
iteration: 10 25.62367011769231 30.4165470621956 1 0.3153782000299543
iteration: 15 21.386872681529344 24.97318062362149 1 0.4819001000141725
iteration: 25 26.41368222626042 24.583275558866166 1 0.6714170000050217
iteration: 39 20.988163289660704 22.896411977254925 1 0.8787903999909759
iteration: 63 13.322090064727783 19.658500865313083 1 1.0605166000314057
iteration: 100 8.635362439929265 12.111157314338307 1 1.2710925999563187
iteration: 158 3.175244549856072 8.482114696371413 1 1.4613897999515757
iteration: 251 2.507320708698731 4.887417985870123 1 1.630535800009966
iteration: 398 2.7184563692865864 3.732978126671068 1 1.8819846999831498
iteration: 630 3.5971262322872746 2.2878312234928204 1 2.1597

In [2]:
d = 10
N = [int(10**(i/5))for i in range(1,36)]
s = 0.6
gamma0 =0.5
t = 0.05
theta = 1/(2*s+1)
Sum_epoch = 10000001
t_kernel_SGD = T_kernel_SGD(d,s)

t_kernel_SGD.updating(gamma0,t,theta,Sum_epoch,N)


iteration: 1 91.90475177622798 111.04990585292896 0 0.005792199983261526
iteration: 2 78.28668734506876 90.25253932401709 0 0.05324010003823787
iteration: 3 64.2489350596762 80.22297054568321 0 0.10362650000024587
iteration: 6 124.79168598861841 61.047197305171856 1 0.1477493999991566
iteration: 10 49.26887092210467 51.87005024795193 1 0.27137880004011095
iteration: 15 35.29678057605507 47.72640923225806 1 0.3894972000271082
iteration: 25 37.69463009624149 43.948318180017225 1 0.5090070000151172
iteration: 39 25.414463755775248 39.92602129957758 1 0.6278937000315636
iteration: 63 11.216931214894545 23.355707079193262 1 0.7425627000629902
iteration: 100 9.46366216171009 16.533487931209823 1 0.8576182000106201
iteration: 158 9.528166886796416 12.903512160197712 1 0.9759954999899492
iteration: 251 7.692570201693267 8.932522201007014 2 1.105847500031814
iteration: 398 3.6002762099185475 6.751096610433781 2 1.6977245000889525
iteration: 630 1.3002880791839317 4.418515942384372 2 2.154601400

In [3]:
A= [[0],[1],[2]]
B = [4]
A.insert(0,B)

In [8]:
a = np.array([0,1,2])
b = np.array([3,4])
np.append(b,a)

array([3, 4, 0, 1, 2])